In [1]:
import pandas as pd
import numpy as np
import os
import subprocess
import matplotlib.pyplot as plt
import statsmodels
%matplotlib inline
plt.style.use("dark_background")

In [2]:
print(os.getcwd())
print(os.listdir(os.getcwd() + "/data/"))


/home/chris/Documents/bigforecast/sandbox/GDELT_FeatureEngineering/GDELT
['20170718.export.CSV', '20170718.CSV.zip']


In [3]:
with open("feature_names.txt", "r") as f:
    cols = f.readlines()

cols = list(map(lambda s: s.replace("\n", ""), cols))
print(cols)

['GlobalEventID', 'Day', 'MonthYear', 'Year', 'FractionDate', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code', 'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode', 'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code', 'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_Fullname', 'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_ADM2Code', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_Fullname', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_ADM2Code', 'Actor2Geo_Lat', 'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type', 'ActionGeo_Fullname', 'ActionGeo_CountryCode',

In [5]:
#gdelt = pd.read_csv("data/20170718201500.export.CSV", sep = "\t", header = None)
gdelt = pd.read_csv("data/20170809053000.export.CSV", sep = "\t", header = None)
gdelt.columns = cols
gdelt.head()

   GlobalEventID       Day  MonthYear  Year  FractionDate Actor1Code  \
0      679808183  20160809     201608  2016        2016.6        NaN   
1      679808184  20160809     201608  2016        2016.6        NaN   
2      679808185  20160809     201608  2016        2016.6        ARE   
3      679808186  20160809     201608  2016        2016.6     ARECOP   
4      679808187  20160809     201608  2016        2016.6     CHNBUS   

             Actor1Name Actor1CountryCode Actor1KnownGroupCode  \
0                   NaN               NaN                  NaN   
1                   NaN               NaN                  NaN   
2  UNITED ARAB EMIRATES               ARE                  NaN   
3                 DUBAI               ARE                  NaN   
4                 CHINA               CHN                  NaN   

  Actor1EthnicCode                        ...                          \
0              NaN                        ...                           
1              NaN      

In [10]:
def extract_value(row):
    """
    Takes the interesting things out of a row in a dataframe and returns
    a dictionary of them
    """
    cols_to_keep = ["GlobalEventID", "Day", "MonthYear", "DATEADDED", "SOURCEURL", "NumMentions", "NumSources", "NumArticles", "AvgTone", "GoldsteinScale", "EventRootCode"]
    return dict(row[cols_to_keep])

list(gdelt.apply(extract_value, 1))[0:3]

[{'AvgTone': -5.17241379310345,
  'DATEADDED': 20170809053000,
  'Day': 20160809,
  'EventRootCode': 19,
  'GlobalEventID': 679808183,
  'GoldsteinScale': -10.0,
  'MonthYear': 201608,
  'NumArticles': 2,
  'NumMentions': 2,
  'NumSources': 1,
  'SOURCEURL': 'http://www.zawya.com/mena/en/story/Dubai_Police_launch_drive_to_make_emirates_beaches_safer-GN_08082017_090824/'},
 {'AvgTone': -2.94695481335953,
  'DATEADDED': 20170809053000,
  'Day': 20160809,
  'EventRootCode': 2,
  'GlobalEventID': 679808184,
  'GoldsteinScale': 3.0,
  'MonthYear': 201608,
  'NumArticles': 3,
  'NumMentions': 3,
  'NumSources': 1,
  'SOURCEURL': 'https://www.reviewjournal.com/opinion/editorials/editorial-protecting-the-rural-west-from-the-environmental-judicial-complex/'},
 {'AvgTone': -1.21212121212121,
  'DATEADDED': 20170809053000,
  'Day': 20160809,
  'EventRootCode': 3,
  'GlobalEventID': 679808185,
  'GoldsteinScale': 4.0,
  'MonthYear': 201608,
  'NumArticles': 4,
  'NumMentions': 4,
  'NumSources': 1

In [ ]:
gdelt.shape

In [ ]:
prices = pd.read_csv("~/Documents/bigforecast/oil_price.csv", sep = "\t")
prices.columns = ["Date", "Price"]
prices["Date"] = pd.to_datetime(prices["Date"])
prices.tail()

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from datetime import date, timedelta
from get_GDELT import get_file

In [ ]:
def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [ ]:
start = date(2017, 7, 1)
end = date(2017, 7, 4)
from get_GDELT import get_file
for single_date in date_range(start, end):
    get_file(file_date = single_date, version = 1, time = None)

In [ ]:
d = date(2017, 7, 18)
print(d.isoformat())
print(type(d.isoformat()))
print(type(d.isoformat()))

In [ ]:
df = pd.concat([pd.read_csv("data/" + file_name,
                            sep = "\t",
                            compression = "zip",
                            low_memory = False,
                            names = cols) for file_name in os.listdir(os.getcwd() + "/data/")])
df.shape


In [ ]:
use_cols = ["Day", "GoldsteinScale"]
avg_days = df[use_cols].groupby("Day").agg(["mean", "median", "var"])
avg_days.columns.values
avg_days.reset_index(inplace = True)

avg_days.columns = ["Date", "mean", "median", "var"]
avg_days["Date"] = pd.to_datetime(avg_days["Date"].astype('str'))

#avg_days.set_index("Day", inplace = True)
avg_days.head()


In [ ]:
avg_days.plot(x = "Day", y = "mean", kind = "bar")

In [ ]:
oil_price = pd.read_csv("~/Documents/bigforecast/oil_price.csv",
                        sep = "\t",
                        names = ["Date", "Price"],
                        parse_dates = ["Date"])
oil_price.head()

In [ ]:
avg_days.head()

In [ ]:
oil_price.tail(10)

In [ ]:
avg_days.join(oil_price, on = "Date", how = "inner", lsuffix = "_left")